# EXPLORATION 12

## Step 1. 데이터 수집하기

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
path_to_dataset = os.getenv("HOME")+"/aiffel/transformer_chatbot/data/ChatbotData.csv"

read = pd.read_csv(path_to_dataset)

In [3]:
data = read[0:10]
len(read)
data.head(100)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


## Step 2. 데이터 전처리하기

SOYNLP를 이용한 단어 토큰화를 하였습니다.
출처:https://wikidocs.net/92961

In [4]:
pip install soynlp

Note: you may need to restart the kernel to use updated packages.


In [5]:
from konlpy.tag import Okt
tokenizer = Okt()
for i in range(10):
    print(tokenizer.morphs(read["Q"][i]))

['12시', '땡', '!']
['1', '지망', '학교', '떨어졌어']
['3', '박', '4일', '놀러', '가고', '싶다']
['3', '박', '4일', '정도', '놀러', '가고', '싶다']
['PPL', '심하네']
['SD', '카드', '망가졌어']
['SD', '카드', '안', '돼']
['SNS', '맞팔', '왜', '안', '하지', 'ㅠㅠ']
['SNS', '시간', '낭비', '인', '거', '아는데', '매일', '하는', '중']
['SNS', '시간', '낭비', '인데', '자꾸', '보게', '됨']


데이터 개수 확인

In [6]:
questions_list = []
answers_list = []
for i in range(len(read)):
    questions_list.append(tokenizer.morphs(read["Q"][i]))
    answers_list.append(tokenizer.morphs(read["A"][i]))

print("전체 샘플 수: ",len(questions_list))
print("전체 샘플 수: ",len(answers_list))

전체 샘플 수:  11823
전체 샘플 수:  11823


In [7]:
questions = []
answers = []

for i in range(len(questions_list)):
    q_temp = ""
    for j in range(len(questions_list[i])-1):
        q_temp += questions_list[i][j] + " "
    q_temp += (questions_list[i][-1])
    questions.append(q_temp)
    
for i in range(len(answers_list)):
    a_temp = ""
    for j in range(len(answers_list[i])-1):
        a_temp += answers_list[i][j] + " "
    a_temp += (answers_list[i][-1])
    answers.append(a_temp)

        

for i in range(10):
    print(questions[i])
    print(answers[i])
    print("="*10)

12시 땡 !
하루 가 또 가네요 .
1 지망 학교 떨어졌어
위로 해 드립니다 .
3 박 4일 놀러 가고 싶다
여행 은 언제나 좋죠 .
3 박 4일 정도 놀러 가고 싶다
여행 은 언제나 좋죠 .
PPL 심하네
눈살 이 찌푸려지죠 .
SD 카드 망가졌어
다시 새로 사는 게 마음 편해요 .
SD 카드 안 돼
다시 새로 사는 게 마음 편해요 .
SNS 맞팔 왜 안 하지 ㅠㅠ
잘 모르고 있을 수도 있어요 .
SNS 시간 낭비 인 거 아는데 매일 하는 중
시간 을 정 하고 해보세요 .
SNS 시간 낭비 인데 자꾸 보게 됨
시간 을 정 하고 해보세요 .


## Step 3. SubwordTextEncoder 사용하기

한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다. 하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인 SubwordTextEncoder를 그대로 사용해보세요.

In [8]:
import tensorflow_datasets as tfds
print("살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐")

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성. (Tensorflow 2.3.0 이상) (클라우드는 2.4 입니다)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
print("슝=3 ")

살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐
슝=3 


시작토큰과 종료토큰 부여

In [9]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("슝=3")

슝=3


In [10]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [7289]
END_TOKEN의 번호 : [7290]


In [11]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

7291


In [12]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [4000, 264, 923, 3051]
정수 인코딩 후의 21번째 답변 샘플: [379, 200, 168, 29, 1888, 336, 1]


In [13]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 40
print(MAX_LENGTH)

40


In [14]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs
print("슝=3")

슝=3


In [15]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 7291
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


### 교사강요

In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
print("슝=3")

슝=3


## Step 4. 모델 구성하기

위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

### 포지셔널 인코딩 레이어

In [17]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

print("슝=3")

슝=3


### scaled_dot_product_attention 함수

In [18]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

print("슝=3")

슝=3


### MultiHeadAttention

In [19]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs
print("슝=3")

슝=3


### 패딩 마스크

In [20]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]
print("슝=3")

슝=3


### look_ahead_mask

In [21]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)
print("슝=3")

슝=3


In [22]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

슝=3


In [23]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
print("슝=3")

슝=3


In [24]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
print("슝=3")

슝=3


In [25]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
print("슝=3")

슝=3


### Transformer

In [26]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
print("슝=3")

슝=3


In [27]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 512 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 512)    6888960     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

### 손실함수

In [28]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)
print("슝=3")

슝=3


### 커스텀 학습률

In [29]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
print("슝=3")

슝=3


In [30]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
print("슝=3")

슝=3


In [31]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/50
185/185 [==============================] - 24s 84ms/step - loss: 1.5589 - accuracy: 0.0309
Epoch 2/50
185/185 [==============================] - 16s 84ms/step - loss: 1.1522 - accuracy: 0.0514
Epoch 3/50
185/185 [==============================] - 16s 87ms/step - loss: 1.0085 - accuracy: 0.0606
Epoch 4/50
185/185 [==============================] - 16s 86ms/step - loss: 0.9098 - accuracy: 0.0696
Epoch 5/50
185/185 [==============================] - 16s 87ms/step - loss: 0.8215 - accuracy: 0.0772
Epoch 6/50
185/185 [==============================] - 16s 87ms/step - loss: 0.7384 - accuracy: 0.0849
Epoch 7/50
185/185 [==============================] - 16s 88ms/step - loss: 0.6563 - accuracy: 0.0938
Epoch 8/50
185/185 [==============================] - 16s 88ms/step - loss: 0.5711 - accuracy: 0.1047
Epoch 9/50
185/185 [==============================] - 16s 88ms/step - loss: 0.4835 - accuracy: 0.1167
Epoch 10/50
185/185 [==============================] - 16s 89ms/step - loss: 0.396

## Step 4. 모델 평가하기

Step 2에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.



### Step 2 에서 사용한 SOYNLP 사용. 출처:https://wikidocs.net/92961

In [32]:
def preprocess_sentence(sentence):
    tokenizer = Okt()
    stc = tokenizer.morphs(sentence)
    
    temp = ""
    for i in range(len(stc)-1):
        temp += stc[i] + " "
    
    temp += stc[-1]
    
    return temp

In [33]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)
print("슝=3")

슝=3


### 띄어쓰기 잘 하기 추가. 출처: https://wikidocs.net/92961

In [34]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-7hqws08x
  Running command git clone --filter=blob:none -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-7hqws08x
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit d8f2b1cb9c904e5a6a181bbf6aeee44fc6e9eab1
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 460.4 MB 10 kB/s               �███████████▋  | 426.2 MB 151.1 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 117.1 MB/s            
     |████████████████████████████████| 462 kB 79.9 MB/s            
     |████████████████████████████████| 1.2 MB 61.3 MB/s            
     |████████████████████████████████| 4.0 MB 98.6 MB/s            
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268397 sha256=45d938119e7c5d13f31a35ef67437d658731037b75290e9f5ecf35a98a736174
  Stored in directory: /tmp/pip-ephem-wheel-cache-p1d7jahn/wheels/ca/f2/8d/94e29f5

In [35]:
from pykospacing import Spacing

In [36]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])
  
  predicted_sentence = predicted_sentence.replace(" ",'') # 띄어쓰기 없애기
  spacing = Spacing()
  predicted_sentence = spacing(predicted_sentence)

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence
print("슝=3")

슝=3


In [53]:
stc = ""
while stc != " ":
    stc = str(input("입력: "))
    if stc == " ":
        print("대화 끝.")
        break
    else:
        sentence_generation(stc)
        print("")

입력: 안녕
입력 : 안녕
출력 : 안녕하세요.

입력: 날 응원해줘
입력 : 날 응원해줘
출력 : 저도 같이 응원할게요.

입력: 고마워
입력 : 고마워
출력 : 감사합니다.

입력: 너는 누구야?
입력 : 너는 누구야?
출력 : 저는 마음을 이어주는 위로 봇입니다.

입력: 졸려
입력 : 졸려
출력 : 오늘 일찍 주무세요.

입력: 그래야지.. 잘자
입력 : 그래야지.. 잘자
출력 : 잠깐 쉬어가도 괜찮아요.

입력: 그래 다음에 또 보자.
입력 : 그래 다음에 또 보자.
출력 : 괜찮은 척 하는 걸 수도 있어요.

입력: 잘있어
입력 : 잘있어
출력 : 잘 살고 있을 거예요.

입력:  
대화 끝.


# 회고

## 이번 프로젝트에서 어려웠던 점.
#### 우선, 전처리 영역이 가장 어려웠습니다.
- 영어 전처리는 어느정도 이해가 되었는데, 한글 전처리 영역은 매우 어려웠습니다. 어떤 부분에서 어떻게 끊어야 하는지도 어려웠지만, 영어의 경우 a-z같은 부분을 한글에서는 파이썬에서 어떻게 표현해야하는가에 대해 어려웠습니다. 띄어쓰기도 영어처럼 단어 별로 끊지 않고, 상황별로 띄어쓰기의 방법이 매우 달랐기 때문에, 이것을 어떻게 처리를 해 주어야 하는가에 대해 정말 많은 골 머리를 앓았습니다. 그러던 중 괜찮은 라이브러리를 찾게되었고, 사실 라이브러리 참고 없이 혼자 함수를 만들어 내어 하려다가, 결국 역시 만들어진 라이브러리가 최고다 라는 생각에 응용하게 되었는데, 생각보다 매우 만족스러운 결과를 얻게되어, 나중에 띄어쓰기 부분까지 참고하여 추가하였습니다.

#### 다음으로는, 내용적인 부분이 어려웠습니다.
- 이전에, 네이버 영화 리뷰와는 다르게, 멀티헤드, 스케일드 닷 프로덕트, 트랜스포머 등 새로이 배운 영역이 추가 되었는데, 이론적으로 완벽하게 이해하기는 어려웠습니다. 하지만 저의 챗 봇이 생각보다 매우 괜찮은 결과를 보여주었기에, 앞으로도 관련된 영역이 있다면, 다시 돌아와 틈틈히 보고 잘 이해하고 넘어 갈 예정입니다.

## 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.
- 알아낸 점으로는, 한국어 전처리 관련 라이브러리에 대해 알게 된 점이 제가 전처리 부분에서 고민했던 만큼 제게 가장 큰 수익이었던것 같습니다. 좀 더 깊게 공부하게 되었을 때에는, 전처리 함수까지 혼자서 구현하고 싶지만, 역시 한글 관련해서 공부하신 학자들과 프로그래머들이 만들어 주신 라이브러리는 훌륭하다는 것을 깨달았습니다. 위의 라이브러리 덕에 아주 매끄러운 채팅 봇을 만들어 낸 것 같아 매우 기쁩니다.
- 하지만 위에 설명 드렸듯, 이번 단원에서 새로이나온 멀티헤드, 스케일드 닷 프로덕트, 트랜스포머, 혹은 패딩 마스크, 더 나아가 커스텀 학습률 등에 대한 이론적인 부분을 잘 캐치하지 못하였기 때문에, 향후에 계속해서 공부해 나갈 예정입니다. 이런 함수를 이곳에서 배울 수 있게 되어 덕분에 좋은 결과를 낸 것 같아 참으로 기쁩니다.

## 루브릭 평가 지표를 맞추기 위해 시도한 것들.
#### 공백과 특수문자 처리, 토크나이징, 병렬 데이터 구축의 과정이 적절히 진행되었다.
- 사실 제가 직접 디테일하게 구분한 것은 없지만, 맞추기 위해 매우 고심하였고, 끝끝내 좋은 적절한 라이브러리를 찾게 되었습니다.

#### 구현한 트랜스포머 모델이 한국어 병렬 데이터 학습시 안정적으로 수렴하였다.
- 위의 모호한 점에, 이론적으로 파악하지 못하였다고 설명했는데, 병렬적 이라는 단어는 알지만, 병렬 데이터 라는 개념에 대해 잘 이해하지 못하였기 때문에 제가 구조적으로 잘 만들어 낸 부분은 자신이 없습니다. 하지만, 학습이 안정적으로 되었기 때문에, 봇이 적절한 대화를 주고 받았다고 생각합니다.

#### 한국어 입력문장에 그럴듯한 한국어로 답변을 리턴하였다.
- 정말 신기할 정도였습니다. 제가 인터넷에서 돌아다니는 채팅봇과 대화를 하는 기분이었습니다. 그 기분을 여기 Jupyter notebook 환경에서 대화를 하고 있으니, 정말 제가 처음으로 저의 첫 인공지능 로봇을 만들어 낸 듯한 흡사 그러한 기분을 들게끔 하여 애정이 많이 갑니다.

## 루브릭 평가 지표를 달성하지 못했을 때, 이유에 관한 추정
- 아무래도, 이론적으로 이해하지 못하였기 때문에, 실습 노드의 함수들을 모두 끌어다 사용하였고, 때문에 혹시나 구조적으로 바꿔주면 좀 더 좋은 결과나 좀 더 깔끔한 결과가 나오는지, 혹은 더 추가할 수 있는데 추가하지 못하였다던지 매끄럽지 못하다 던지에 대한 판단을 스스로 할 수가 없었습니다. 솔직하게 제가 심히 고민하고 해결한 부분은 전처리 부분이 전부이며, 나머지 부분은 이해하려 시도하였으나, 확실한 결과를 얻어내지 못하였기 때문에, 루브릭 지표를 달성하지 못하였다면 이부분이 아닐까 추정해 봅니다.

## 자기 다짐
- 비록 내용적인 부분에서 다소 답답함을 느끼고 결국 완전히 습득하지는 못한 채로 노드를 완성하였지만, 결과가 제 생각 이상으로 좋은 결과를 보여 주었기에, 스스로가 매우 뿌듯하여 자신감이 생기는 노드였습니다. 앞에 두,세개 전 노드들은, 거의 반성문을 쓰다 시피 매우 안좋은 마음으로 끝이 났는데, 그 때 느낀 다짐으로, 욕심내거나 완벽하게, 그리고 모든걸 알아내려고 하지말고, 지금 내가 할 수 있는 이 상황에서 최선을 다하고 단 하나만 얻어가도 기분좋게 얻어가자 라는 다짐 덕분인지는 모르겠지만, 이전 노드들에서 느낀점의 연장선으로, 이번 노드에도 얻은것이 많아 기쁩니다. 위에 언급하였듯, 아직 모호하고 이해하지 못한 부분이 대부분 이지만, 알아낸 부분도 많고 무엇보다 자신감을 얻었기에, 많이 배우고 가며, 앞으로 공부할 내용이 더 있다는 사실에도 기쁩니다. 계속해서 지금같은 마음가짐으로 열심히 하겠습니다.

감사합니다.